In [1]:
import pandas as pd
import numpy as np
import warnings
from typing import Dict, Iterator, Any
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
import os
from tqdm.auto import tqdm
# register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
tqdm.pandas()
from enum import StrEnum
import math
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
class OrigDataSchema(StrEnum):
    IS_RETWEET = "is_retweet"
    TIMESTAMP = "tweetcreatedts"
    ID = "tweetid"
    TEXT = "text"

    IS_DUPL = "is_duplicate"


class HashedDataSchema(StrEnum):
    HASH = "hash"
    ID = "tweetid"
    TIMESTAMP = "tweetcreatedts"
    DATE = "date"

def iterate_dataframes(path: str) -> Iterator[pd.DataFrame]:
    """
    Iterates over all .csv files in path as pd.DataFrame
    """
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
        csvs = [path + x for x in os.listdir(path) if "csv" in x]
    
        for csv in tqdm(csvs):
            yield pd.read_csv(csv,  lineterminator='\n')

def aggregate_dataframe(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Aggregates dataframe to dict.
    """
    df_la = df.groupby("language").count().reset_index()
    languages = df_la['language'].tolist()
    lang_counts = df_la['username'].tolist()
    df['dupl'] = df.duplicated(subset="text")
    languages_dupl = [la + "_dupl" for la in languages]
    lang_dupl_counts = df.groupby("language")['dupl'].sum().tolist()
    unique_user_count = df['userid'].unique().shape[0]
    row_count = df.shape[0]
    duplicated_count = df.duplicated(subset="text").sum()
    date = df.iloc[0]['tweetcreatedts'][:10]
    aggregation = {'unique_users': unique_user_count,
                  'row_count': row_count,
                  'text_duplicated_count': duplicated_count,
                  'date':date}
    aggregation = {**dict(zip(languages, lang_counts)), **aggregation, **dict(zip(languages_dupl, lang_dupl_counts))}
    return aggregation
    


    

def get_all_aggregated_data() -> pd.DataFrame:
    """
    Loads or creates all aggregated data.
    """
    path_2023 = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/2023_agg.csv"
    path_2022 = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/2022_agg.csv"

    df_agg_2023 = pd.read_csv(path_2023)
    df_agg_2022 = pd.read_csv(path_2022)

    df_agg = df_agg = pd.concat([df_agg_2022,df_agg_2023]).fillna(0)
    return df_agg


def create_hashes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates dataframes containing only hashes and tweetids from not duplicates.
    """
    if OrigDataSchema.IS_RETWEET in df.columns:
        df = df[~df.duplicated(subset=OrigDataSchema.TEXT) & ~df[OrigDataSchema.IS_RETWEET]]
    else:
        df =df[~df.duplicated(subset=OrigDataSchema.TEXT)]
        
    df_hashed = df[[OrigDataSchema.TEXT, OrigDataSchema.ID,OrigDataSchema.TIMESTAMP]].copy()
    
    df_hashed[HashedDataSchema.HASH] = df_hashed[OrigDataSchema.TEXT].apply(hash)
    
    
    df_hashed[HashedDataSchema.DATE] = df_hashed[OrigDataSchema.TIMESTAMP].apply(lambda ts: pd.to_datetime(ts[:10]))

    df_hashed = df_hashed.drop(OrigDataSchema.TEXT, axis=1).set_index(HashedDataSchema.ID).drop(OrigDataSchema.TIMESTAMP, axis=1)
    return df_hashed


def aggregate_hash_data(dir_path: str, target_path: str) -> pd.DataFrame:
    """
    Creates aggregated data frame and saves it as csv.
    """

    hashed_dfs = []
    for df in iterate_dataframes(dir_path):
        hashed = create_hashes(df)
        hashed_dfs.append(hashed)

    hashed_df = pd.concat(hashed_dfs)
    hashed_df.to_csv(target_path)

def get_all_hashed_data() -> pd.DataFrame:
    """
    Loads all hashed dataframes.
    """
    target_2022 = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/AggregatedData/2022_hashed.csv"
    target_2023 = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/AggregatedData/2023_hashed.csv"

    df_hash_2023 = pd.read_csv(target_2022)
    df_hash_2022 = pd.read_csv(target_2023)

    df_hashed  = pd.concat([df_hash_2022,df_hash_2023]).sort_values("date")
    return df_hashed

def iterate_dataframes_path(path: str) -> tuple[Iterator[pd.DataFrame],str]:
    """
    Iterates over all .csv files in path as pd.DataFrame
    """
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
        csvs = [path + x for x in os.listdir(path) if "csv" in x]
    
        for csv in tqdm(csvs):
            yield pd.read_csv(csv,  lineterminator='\n'), csv


def add_is_dupl_to_data() -> None:
    path_2022_data = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/2022/"
    path_2023_data = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/2023/"

    df_hashed = get_all_hashed_data()
    df_hashed = df_hashed[~df_hashed.duplicated('hash')]
    
    for df, path in iterate_dataframes_path(path_2022_data):
        df[OrigDataSchema.IS_DUPL] = df[OrigDataSchema.ID].isin(df_hashed[HashedDataSchema.ID])
        df.to_csv()

In [19]:
path_2022_data = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/2022/"
target_2022 = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/AggregatedData/2022_hashed.csv"
path_2022_data = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/2022/"okodil
target_2023 = "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/AggregatedData/2023_hashed.csv"
#aggregate_hash_data(path_2022_data, target_2022)
#aggregate_hash_data(path_2023_data, target_2023)
df = get_all_hashed_data()
print(df.shape)
df = df[~df.duplicated('hash')]
print(df.shape)

(30110382, 3)
(28498970, 3)


In [20]:
df_t = next(iterate_dataframes(path_2022_data))

df_t[OrigDataSchema.IS_DUPL] = df_t[OrigDataSchema.ID].isin(df[HashedDataSchema.ID])

  0%|                                                   | 0/311 [00:03<?, ?it/s]


In [29]:
df_t[OrigDataSchema.IS_DUPL].sum()/df_t.shape[0]

0.3751567150682496

In [27]:
df_t['is_duplicate'].sum()

122387

In [28]:
df_t.shape

(326229, 30)

In [16]:
df_t.columns

Index(['Unnamed: 0', 'userid', 'username', 'acctdesc', 'location', 'following',
       'followers', 'totaltweets', 'usercreatedts', 'tweetid',
       'tweetcreatedts', 'retweetcount', 'text', 'hashtags', 'language',
       'coordinates', 'favorite_count', 'is_retweet', 'original_tweet_id',
       'original_tweet_userid', 'original_tweet_username',
       'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_screen_name', 'is_quote_status', 'quoted_status_id',
       'quoted_status_userid', 'quoted_status_username', 'extractedts'],
      dtype='object')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30110382 entries, 1525264628711936000 to 1631081855142318083
Data columns (total 2 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tweetcreatedts  object
 1   hash            int64 
dtypes: int64(1), object(1)
memory usage: 689.2+ MB


In [87]:
df

,hash
tweetid,
1525264628711936000,7512023704103795790
1525264628715880449,-1632070416298900772
1525264628925624325,8837051791152875786
1525264628934234120,-8947177930304202717
1525264629626191880,-1186198326629031726
...,...
1631081836112642048,7417735237431925445
1631081844127965185,4297768191860994750
1631081845906448387,3482933603353950004
